In [1]:
from transformers import AutoTokenizer
import os
from os import listdir
from os.path import isfile, join
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import xml.etree.ElementTree as ET
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
import spacy
nlp = spacy.load("en_core_web_sm")
from eventseg_getter import *
from tense_tagger import *
import requests

In [2]:
def tense_getter(txt):
    return print_parsed_text(txt)

space = ' '

#model = RobertaModel.from_pretrained('roberta-base')
#dir_name = "/shared/why16gzl/logic_driven/Quizlet/Quizlet_2/LDC2020E20_KAIROS_Quizlet_2_TA2_Source_Data_V1.0/data/ltf/ltf/"
#file_name = "K0C03N4LR.ltf.xml"    # Use ltf_reader 
#dir_name = "/home1/w/why16gzl/KAIROS/hievents_v2/processed/"
#file_name = "article-10901.tsvx"   # Use tsvx_reader

# ============================
#         PoS Tagging
# ============================
pos_tags = ["ADJ", "ADP", "ADV", "AUX", "CONJ", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "PUNCT", "SCONJ", "SYM", "VERB", "X", "SPACE"]
identity_matrix = np.identity(len(pos_tags))
postag_to_OneHot = {}
postag_to_OneHot["None"] = np.zeros(len(pos_tags))
for (index, item) in enumerate(pos_tags):
    postag_to_OneHot[item] = identity_matrix[index]
    
def postag_2_OneHot(postag):
    return postag_to_OneHot[postag]

# ===========================
#        HiEve Labels
# ===========================

label_dict={"SuperSub": 0, "SubSuper": 1, "Coref": 2, "NoRel": 3}
num_dict = {0: "SuperSub", 1: "SubSuper", 2: "Coref", 3: "NoRel"}
def label_to_num(label):
    return label_dict[label]
def num_to_label(num):
    return num_dict[num]

# Padding function, both for huggingface encoded sentences, and for part-of-speech tags
def padding(sent, pos = False, max_sent_len = 200):
    if pos == False:
        one_list = [1] * max_sent_len
        one_list[0:len(sent)] = sent
        return torch.tensor(one_list, dtype=torch.long)
    else:
        one_list = ["None"] * max_sent_len
        one_list[0:len(sent)] = sent
        return one_list

def transformers_list(content, tokenizer, token_list = None, token_span_SENT = None):
    #tokenizer = AutoTokenizer.from_pretrained(transformers_model)    
    encoded = tokenizer.encode(content)
    # input_ids = torch.tensor(encoded).unsqueeze(0)  # Batch size 1
    # outputs = model(input_ids)
    # last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
    _subwords = []
    _subword_to_ID = []
    _subwords_no_space = []
    for index, i in enumerate(encoded):
        r_token = tokenizer.decode([i])
        if len(r_token) > 0:
            _subword_to_ID.append(i)
            _subwords.append(r_token)
            if r_token[0] == " ":
                _subwords_no_space.append(r_token[1:])
            else:
                _subwords_no_space.append(r_token)

    _subword_span = tokenized_to_origin_span(content, _subwords_no_space[1:-1]) # w/o <s> and </s>
    _subword_map = []
    if token_span_SENT is not None:
        _subword_map.append(-1) # "<s>"
        for subword in _subword_span:
            _subword_map.append(token_id_lookup(token_span_SENT, subword[0], subword[1]))
        _subword_map.append(-1) # "</s>" 
        return _subword_to_ID, _subwords, _subword_span, _subword_map
    else:
        return _subword_to_ID, _subwords, _subword_span, -1

def tokenized_to_origin_span(text, token_list):
    token_span = []
    pointer = 0
    previous_pointer = 0
    for token in token_list:
        while pointer < len(text):
            if token[0] == text[pointer]:
                start = pointer
                end = start + len(token) - 1
                previous_pointer = pointer = end + 1
                break
            else:
                pointer += 1
        if pointer < len(text):
            token_span.append([start, end])
        else:
            if previous_pointer < len(text):
                # exceeding text length, meaning that a weird character is encountered
                token_span.append([end + 1, end + 1])
                pointer = previous_pointer
            else:
                # end of text
                token_span.append([start, end])
    return token_span

def sent_id_lookup(my_dict, start_char, end_char = None):
    for sent_dict in my_dict['sentences']:
        if end_char is None:
            if start_char >= sent_dict['sent_start_char'] and start_char <= sent_dict['sent_end_char']:
                return sent_dict['sent_id']
        else:
            if start_char >= sent_dict['sent_start_char'] and end_char <= sent_dict['sent_end_char']:
                return sent_dict['sent_id']

def token_id_lookup(token_span_SENT, start_char, end_char):
    for index, token_span in enumerate(token_span_SENT):
        if start_char >= token_span[0] and end_char <= token_span[1]:
            return index

def span_SENT_to_DOC(token_span_SENT, sent_start):
    token_span_DOC = []
    #token_count = 0
    for token_span in token_span_SENT:
        start_char = token_span[0] + sent_start
        end_char = token_span[1] + sent_start
        #assert my_dict["doc_content"][start_char] == sent_dict["tokens"][token_count][0]
        token_span_DOC.append([start_char, end_char])
        #token_count += 1
    return token_span_DOC

def id_lookup(span_SENT, start_char):
    # this function is applicable to huggingface subword or token from ltf/spaCy
    # id: start from 0
    token_id = -1
    for token_span in span_SENT:
        token_id += 1
        if token_span[0] <= start_char and token_span[1] >= start_char:
            return token_id
    raise ValueError("Nothing is found.")
    return token_id

def segment_id_lookup(segments, sent_id):
    for i in range(len(segments)):
        if sent_id > segments[i] and sent_id <= segments[i+1]:
            return i
"""
map_tense = {"futu": "Future",
             "perf": "Perfect",
             "simp": "Simple", 
             "cont": "Continuous",
             "past": "Past",
             "pres": "Present",
             "pass": "Passive"
            }
def tense_mapper(tense):
    tense = list(map(''.join, zip(*[iter(tense)]*4))) 
    return ' '.join([map_tense[i] for i in tense])
"""
map_tense = {"futu": "Future",
             "perf": "Perfect",
             #"simp": "",      # DELETE simp
             "simp": "Simple", # KEEP simp
             "cont": "Continuous",
             "past": "Past",
             "pres": "Present",
             "pass": "Passive",
            }
def tense_mapper(tense):
    tense = list(map(''.join, zip(*[iter(tense)]*4))) 
    return ' '.join([map_tense[i] for i in tense]).strip()
        
def tense_finder(tense_list, start_char):
    # start_char: sentence level 
    # e.g., 'She would say the soldiers were hit by a truck.'
    # ['would', 'MD', [4, 9], 'futusimp']
    # ['say', 'VB', [10, 13], 'futusimp', ...
    for verb in tense_list:
        if verb[2][0] == start_char:
            try:
                return [tense_mapper(verb[3]), '['+verb[3]+']'] 
            # Updated on Oct 29, 2022, because of unknown error
            except:
                return None
            # Updated on Feb 27, 2022
    #print(start_char)
    #print(tense_list)
    return None

In [3]:
def tdd_reader(text, event_pos, event_pos_end, tokenizer):
    my_dict = {}
    my_dict["event_dict"] = {}
    my_dict["sentences"] = []
    my_dict["relation_dict"] = {}
    
    # Read tsvx file
    my_dict["doc_content"] = text
        
    for i, pos in enumerate(event_pos):
        my_dict["event_dict"][i] = {"mention": text[event_pos[i]:event_pos_end[i]], "start_char": event_pos[i], "end_char": event_pos_end[i]} 
    
    # Split document into sentences
    sent_tokenized_text = sent_tokenize(my_dict["doc_content"])
    #sent_tokenized_text = []
    #tense_res = tense_getter(text)
    #for sentence in tense_res['sentences']:
    #    sent_tokenized_text.append(sentence[0])
        
    sent_span = tokenized_to_origin_span(my_dict["doc_content"], sent_tokenized_text)
    count_sent = 0
    end_pos = [1]
    for sent in sent_tokenized_text:
        sent_dict = {}
        sent_dict["sent_id"] = count_sent
        sent_dict["content"] = sent
        sent_dict["sent_start_char"] = sent_span[count_sent][0]
        sent_dict["sent_end_char"] = sent_span[count_sent][1]
        sent_dict["tense_list"] = tense_getter(sent)
        
        spacy_token = nlp(sent_dict["content"])
        sent_dict["tokens"] = []
        sent_dict["pos"] = []
        # spaCy-tokenized tokens & Part-Of-Speech Tagging
        for token in spacy_token:
            sent_dict["tokens"].append(token.text)
            sent_dict["pos"].append(token.pos_)
        sent_dict["token_span_SENT"] = tokenized_to_origin_span(sent, sent_dict["tokens"])
        sent_dict["token_span_DOC"] = span_SENT_to_DOC(sent_dict["token_span_SENT"], sent_dict["sent_start_char"])

        # huggingface tokenizer
        sent_dict["_subword_to_ID"], sent_dict["_subwords"], \
        sent_dict["_subword_span_SENT"], sent_dict["_subword_map"] = \
        transformers_list(sent_dict["content"], tokenizer, sent_dict["tokens"], sent_dict["token_span_SENT"])
        
        if count_sent == 0:
            end_pos.append(len(sent_dict["_subword_to_ID"]))
        else:
            end_pos.append(end_pos[-1] + len(sent_dict["_subword_to_ID"]) - 1)
            
        sent_dict["_subword_span_DOC"] = \
        span_SENT_to_DOC(sent_dict["_subword_span_SENT"], sent_dict["sent_start_char"])
        
        sent_dict["_subword_pos"] = []
        for token_id in sent_dict["_subword_map"]:
            if token_id == -1 or token_id is None:
                sent_dict["_subword_pos"].append("None")
            else:
                sent_dict["_subword_pos"].append(sent_dict["pos"][token_id])
        
        my_dict["sentences"].append(sent_dict)
        count_sent += 1
        
    my_dict['end_pos'] = end_pos
    # Add sent_id as an attribute of event
    for event_id, event_dict in my_dict["event_dict"].items():
        my_dict["event_dict"][event_id]["sent_id"] = sent_id = \
        sent_id_lookup(my_dict, event_dict["start_char"], event_dict["end_char"])
        #my_dict["event_dict"][event_id]["token_id"] = \
        #id_lookup(my_dict["sentences"][sent_id]["token_span_DOC"], event_dict["start_char"]) # updated on Oct 29, 2022, because of unknown error
        my_dict["event_dict"][event_id]["_subword_id"] = \
        id_lookup(my_dict["sentences"][sent_id]["_subword_span_DOC"], event_dict["start_char"]) + 1
        # updated on Mar 20, 2021, plus 1 because of [CLS] or <s>
        my_dict["event_dict"][event_id]["tense"] = tense_finder(my_dict['sentences'][sent_id]["tense_list"], event_dict["start_char"] - my_dict['sentences'][sent_id]["sent_start_char"])
        # updated on Oct 24, 2022, because of change of tense identification service
    return my_dict

def convert_t5_input(text, tl):
    #text = t5_TDD_dic['man-test']['APW19980308.0201'][0][0][19:]
    pointer = text.find("<extra_id_")
    event_pos = []
    event_pos_end = []
    event_ids = []
    e_dict = {}
    while pointer != -1:
        offset = 2
        while text[pointer-offset].isalpha():
            offset += 1
        event_pos.append(pointer-offset+1)
        event_pos_end.append(pointer-1)
        offset = 1
        while text[pointer+10+offset].isdigit():
            offset += 1
        event_id = text[pointer+10:pointer+10+offset]
        event_ids.append(event_id)
        assert text[pointer+10+offset] == '>'
        offset += 1
        if text[pointer+10+offset] == ' ':
            offset += 1
        text = text[0:pointer] + text[pointer+10+offset:]
        pointer = text.find("<extra_id_")
    for i, event_id in enumerate(event_ids):
        e_dict[event_id] = i
    timeline = []
    abnormal = 0
    for i in tl:
        try:
            timeline.append(e_dict[i.replace('<extra_id_', '').replace('>', '')])
        except:
            #print("abnormal")
            abnormal = 1
    return text, event_pos, event_pos_end, event_ids, e_dict, timeline, abnormal







In [29]:
def tml_reader(dir_name, file_name, tokenizer):
    my_dict = {}
    my_dict["event_dict"] = {}
    my_dict["eiid_dict"] = {}
    my_dict["doc_id"] = file_name.replace(".tml", "") 
    # e.g., file_name = "ABC19980108.1830.0711.tml"
    # dir_name = '/shared/why16gzl/logic_driven/EMNLP-2020/MATRES/TBAQ-cleaned/TimeBank/'
    tree = ET.parse(dir_name + file_name)
    root = tree.getroot()
    MY_STRING = str(ET.tostring(root))
    # ================================================
    # Load the lines involving event information first
    # ================================================
    event_id_why = 0
    for makeinstance in root.findall('MAKEINSTANCE'):
        instance_str = str(ET.tostring(makeinstance)).split(" ")
        try:
            assert instance_str[3].split("=")[0] == "eventID"
            assert instance_str[2].split("=")[0] == "eiid"
            eiid = int(instance_str[2].split("=")[1].replace("\"", "")[2:])
            eID = instance_str[3].split("=")[1].replace("\"", "")
        except:
            for i in instance_str:
                if i.split("=")[0] == "eventID":
                    eID = i.split("=")[1].replace("\"", "")
                if i.split("=")[0] == "eiid":
                    eiid = int(i.split("=")[1].replace("\"", "")[2:])
        # Not all document in the dataset contributes relation pairs in MATRES
        # Not all events in a document constitute relation pairs in MATRES
        
        if my_dict["doc_id"] in eiid_to_event_trigger.keys():
            if eiid in eiid_to_event_trigger[my_dict["doc_id"]].keys():
                event_id_why += 1
                my_dict["event_dict"][eID] = {"eiid": eiid, "mention": eiid_to_event_trigger[my_dict["doc_id"]][eiid], "event_id_why": event_id_why}
                my_dict["eiid_dict"][eiid] = {"eID": eID}
        
    # ==================================
    #              Load Text
    # ==================================
    start = MY_STRING.find("<TEXT>") + 6
    end = MY_STRING.find("</TEXT>")
    MY_TEXT = MY_STRING[start:end]
    while MY_TEXT[0] == " ":
        MY_TEXT = MY_TEXT[1:]
    MY_TEXT = MY_TEXT.replace("\\n", " ")
    MY_TEXT = MY_TEXT.replace("\\'", "\'")
    MY_TEXT = MY_TEXT.replace("  ", " ")
    MY_TEXT = MY_TEXT.replace(" ...", "...")
    
    # ========================================================
    #    Load position of events, in the meantime replacing 
    #    "<EVENT eid="e1" class="OCCURRENCE">turning</EVENT>"
    #    with "turning"
    # ========================================================
    while MY_TEXT.find("<") != -1:
        start = MY_TEXT.find("<")
        end = MY_TEXT.find(">")
        if MY_TEXT[start + 1] == "E":
            event_description = MY_TEXT[start:end].split(" ")
            desp_idx = 1
            if not event_description[desp_idx].split("=")[0] == "eid":
                desp_idx = 2
            eID = (event_description[desp_idx].split("="))[1].replace("\"", "")
            MY_TEXT = MY_TEXT[:start] + MY_TEXT[(end + 1):]
            if eID in my_dict["event_dict"].keys():
                my_dict["event_dict"][eID]["start_char"] = start # loading position of events
        else:
            MY_TEXT = MY_TEXT[:start] + MY_TEXT[(end + 1):]
    
    # =====================================
    # Enter the routine for text processing
    # =====================================
    
    my_dict["doc_content"] = MY_TEXT
    my_dict["sentences"] = []
    my_dict["relation_dict"] = {}
    sent_tokenized_text = sent_tokenize(my_dict["doc_content"])
    #sent_tokenized_text = []
    #tense_res = tense_getter(MY_TEXT)
    #for sentence in tense_res['sentences']:
    #    sent_tokenized_text.append(sentence[0])
        
    sent_span = tokenized_to_origin_span(my_dict["doc_content"], sent_tokenized_text)
    end_pos = [1]
    for count_sent, sent in enumerate(sent_tokenized_text):
        sent_dict = {}
        sent_dict["sent_id"] = count_sent
        sent_dict["content"] = sent
        sent_dict["sent_start_char"] = sent_span[count_sent][0]
        sent_dict["sent_end_char"] = sent_span[count_sent][1]
        sent_dict["tense_list"] = tense_getter(sent)
        
        spacy_token = nlp(sent_dict["content"])
        sent_dict["tokens"] = []
        sent_dict["pos"] = []
        # spaCy-tokenized tokens & Part-Of-Speech Tagging
        for token in spacy_token:
            sent_dict["tokens"].append(token.text)
            sent_dict["pos"].append(token.pos_)
        sent_dict["token_span_SENT"] = tokenized_to_origin_span(sent, sent_dict["tokens"])
        sent_dict["token_span_DOC"] = span_SENT_to_DOC(sent_dict["token_span_SENT"], sent_dict["sent_start_char"])

        # huggingface tokenizer
        sent_dict["_subword_to_ID"], sent_dict["_subwords"], \
        sent_dict["_subword_span_SENT"], sent_dict["_subword_map"] = \
        transformers_list(sent_dict["content"], tokenizer, sent_dict["tokens"], sent_dict["token_span_SENT"])
            
        if count_sent == 0:
            end_pos.append(len(sent_dict["_subword_to_ID"]))
        else:
            end_pos.append(end_pos[-1] + len(sent_dict["_subword_to_ID"]) - 1)
            
        sent_dict["_subword_span_DOC"] = \
        span_SENT_to_DOC(sent_dict["_subword_span_SENT"], sent_dict["sent_start_char"])
        
        sent_dict["_subword_pos"] = []
        for token_id in sent_dict["_subword_map"]:
            if token_id == -1 or token_id is None:
                sent_dict["_subword_pos"].append("None")
            else:
                sent_dict["_subword_pos"].append(sent_dict["pos"][token_id])
        
        my_dict["sentences"].append(sent_dict)
        count_sent += 1
        
    my_dict['end_pos'] = end_pos
    # Add sent_id as an attribute of event
    for event_id, event_dict in my_dict["event_dict"].items():
        my_dict["event_dict"][event_id]["sent_id"] = sent_id = \
        sent_id_lookup(my_dict, event_dict["start_char"])
        my_dict["event_dict"][event_id]["token_id"] = \
        id_lookup(my_dict["sentences"][sent_id]["token_span_DOC"], event_dict["start_char"])
        my_dict["event_dict"][event_id]["_subword_id"] = \
        id_lookup(my_dict["sentences"][sent_id]["_subword_span_DOC"], event_dict["start_char"]) + 1 
        # updated on Mar 20, 2021
        #my_dict["event_dict"][event_id]["tense"] = tense_finder(tense_res['sentences'][sent_id][1], event_dict["start_char"] - my_dict['sentences'][sent_id]["sent_start_char"])
        # updated on Feb 21, 2021
        my_dict["event_dict"][event_id]["tense"] = tense_finder(my_dict['sentences'][sent_id]["tense_list"], event_dict["start_char"] - my_dict['sentences'][sent_id]["sent_start_char"])
        # updated on Oct 24, 2022, because of change of tense identification service
        
    return my_dict

In [5]:
mypath_TB = './MATRES_old/TBAQ-cleaned/TimeBank/' # after correction
onlyfiles_TB = [f for f in listdir(mypath_TB) if isfile(join(mypath_TB, f))]
mypath_AQ = './MATRES_old/TBAQ-cleaned/AQUAINT/' 
onlyfiles_AQ = [f for f in listdir(mypath_AQ) if isfile(join(mypath_AQ, f))]
mypath_PL = './MATRES_old/te3-platinum/'
onlyfiles_PL = [f for f in listdir(mypath_PL) if isfile(join(mypath_PL, f))]

In [55]:
# ========================================
#        MATRES: read relation file
# ========================================
# MATRES has separate text files and relation files
# We first read relation files


MATRES_timebank = './MATRES_old/timebank.txt'
MATRES_aquaint = './MATRES_old/aquaint.txt'
MATRES_platinum = './MATRES_old/platinum.txt'
temp_label_map = {"BEFORE": 0, "AFTER": 1, "EQUAL": 2, "VAGUE": 3}
eiid_to_event_trigger = {}
eiid_pair_to_label = {}   

# =========================
#       MATRES Reader
# =========================
def MATRES_READER(matres_file, eiid_to_event_trigger, eiid_pair_to_label):
    with open(matres_file, "r") as f_matres:
        content = f_matres.read().split("\n")
#         assert len(content[-1].split("\t")) == 1
#         content = content[:-1]
        for rel in content:
            rel = rel.split("\t")
            fname = rel[0]
            trigger1 = rel[1]
            trigger2 = rel[2]
            eiid1 = int(rel[3])
            eiid2 = int(rel[4])
            tempRel = temp_label_map[rel[5]]

            if fname not in eiid_to_event_trigger:
                eiid_to_event_trigger[fname] = {}
                eiid_pair_to_label[fname] = {}
            eiid_pair_to_label[fname][(eiid1, eiid2)] = tempRel
            if eiid1 not in eiid_to_event_trigger[fname].keys():
                eiid_to_event_trigger[fname][eiid1] = trigger1
            if eiid2 not in eiid_to_event_trigger[fname].keys():
                eiid_to_event_trigger[fname][eiid2] = trigger2

MATRES_READER(MATRES_timebank, eiid_to_event_trigger, eiid_pair_to_label)
MATRES_READER(MATRES_aquaint, eiid_to_event_trigger, eiid_pair_to_label)
MATRES_READER(MATRES_platinum, eiid_to_event_trigger, eiid_pair_to_label)



# demo.py

In [7]:
import tqdm
import argparse
import time
import datetime
from datetime import datetime 
import random
from matres_reader_with_tense import *
import os
from os import listdir
from os.path import isfile, join
from torch.utils.data import DataLoader
from util import *
from pprint import pprint
from transformers import AutoTokenizer, AutoModel
from model import transformers_mlp_cons
from exp import *
import numpy as np
import json
import sys
from synonyms import *
import pickle
from timeline_construct import *
from ts import func, ModelWithTemperature
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
# datetime object containing current date and time
now = datetime.datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%m/%d/%Y %H:%M:%S")
print("date and time =", dt_string)

#label_dict={"SuperSub": 0, "SubSuper": 1, "Coref": 2, "NoRel": 3}
num_dict = {0: "before", 1: "after", 2: "equal", 3: "vague"}
#def label_to_num(label):
#    return label_dict[label]
def num_to_label(num):
    return num_dict[num]

mask_in_input_ids = 0 # note that [MASK] is actually learned through pre-training
mask_in_input_mask = 0 # when input is masked through attention, it would be replaced with [PAD]
acronym = 0 # using acronym for tense (e.g., pastsimp): 1; else (e.g., past simple): 0
t_marker = 1
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
        
    
def collate_fn(batch):
    max_len = max([len(f['input_ids']) for f in batch])
    input_ids = [f['input_ids'] + [0] * (max_len - len(f['input_ids'])) for f in batch]
    if mask_in_input_ids:
        input_ids_new = []
        for f_id, f in enumerate(input_ids):
            for event_id, start in enumerate(batch[f_id]['event_pos']):
                end = batch[f_id]['event_pos_end'][event_id]
                for token_id in range(start, end): # needs verification
                    f[token_id] = 67
            input_ids_new.append(f)
        input_ids = input_ids_new
    input_ids = torch.tensor(input_ids, dtype=torch.long)
    input_mask = [[1.0] * len(f['input_ids']) + [0.0] * (max_len - len(f['input_ids'])) for f in batch]
    if mask_in_input_mask:
        input_mask_new = []
        for f_id, f in enumerate(input_mask):
            for event_id, start in enumerate(batch[f_id]['event_pos']):
                end = batch[f_id]['event_pos_end'][event_id]
                for token_id in range(start, end): # needs verification
                    f[token_id] = 0.0
            input_mask_new.append(f)
        input_mask = input_mask_new
    # Updated on May 17, 2022    
    input_mask_eo = [[0.0] * max_len for f in batch]
    for f_id, f in enumerate(input_mask_eo):
        for event_id, start in enumerate(batch[f_id]['event_pos']):
            end = batch[f_id]['event_pos_end'][event_id]
            for token_id in range(start, end): # needs verification
                f[token_id] = 1.0
    # Updated on Jun 14, 2022
    input_mask_xbar = [[0.0] * max_len for f in batch]
    input_mask_xbar = torch.tensor(input_mask_xbar, dtype=torch.float)
    input_mask_eo = torch.tensor(input_mask_eo, dtype=torch.float)
    input_mask = torch.tensor(input_mask, dtype=torch.float)
    event_pos = [f['event_pos'] for f in batch]
    event_pos_end = [f['event_pos_end'] for f in batch]
    event_pair = [f['event_pair'] for f in batch]
    labels = [f['labels'] for f in batch]
    output = (input_ids, input_mask, event_pos, event_pos_end, event_pair, labels, input_mask_eo, input_mask_xbar)
    return output


date and time = 03/10/2023 03:16:14


[nltk_data] Downloading package punkt to
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [8]:
f1_metric = 'micro'
params = {'transformers_model': 'google/bigbird-roberta-large',
          'dataset': 'MATRES',   # 'HiEve', 'IC', 'MATRES' 
          'testdata': 'None', # MATRES / MATRES_nd / TDD / PRED / None; None means training mode
          'block_size': 64,
          'add_loss': 0, 
          'batch_size': 1,    # 6 works on 48G gpu
          'epochs': 40,
          'learning_rate': 5e-6,    # subject to change
          'seed': 0,
          'gpu_id': '11453',    # subject to change
          'debug': 0,
          'rst_file_name': '0511pm-lr5e-6-b20-gpu9942-loss0-dataMATRES-accum1-marker@**@-pair1-acr0-tmarker1-td1-dpn1-mask0.rst',    # subject to change
          'mask_in_input_ids': mask_in_input_ids,
          'mask_in_input_mask': mask_in_input_mask,
          'marker': 'abc', 
          'tense_acron': 0, # 1 (acronym of tense) or 0 (original tense)
          't_marker': 1, # 2 (trigger enclosed by special tokens) or 1 (tense enclosed by **)
          'td': 1, # 0 (no tense detection) or 1 (tense detection, add tense info)
          'dpn': 1, # 1 if use DPN; else 0
          'lambda_1': -10, # lower bound * 10
          'lambda_2': 11, # upper bound * 10
          'f1_metric': f1_metric, 
         }
# $acr $tmarker $td $dpn $mask $lambda_1 $lambda_2

# FOR 48GBgpu
if params['testdata'] in ['MATRES', 'MATRES_nd']:
    #params['batch_size'] = 400
    params['batch_size'] = 1
if params['testdata'] in ['TDD']:
    params['batch_size'] = 100
    
if params['testdata'] == 'MATRES_nd':
    params['nd'] = True
else:
    params['nd'] = False
    
###########
# NO MASK #
###########

if params['rst_file_name'] == '0511pm-lr5e-6-b20-gpu9942-loss0-dataMATRES-accum1-marker@**@-pair1-acr0-tmarker1-td1-dpn1-mask0.rst':
    slurm_id = '11453'
#params['rst_file_name'] = '0511pm-lr5e-6-b20-gpu9942-loss0-dataMATRES-accum1-marker@**@-pair1-acr0-tmarker1-td1-dpn1-mask0.rst' 
#slurm_id = '11453'
# python main_pair.py 0615_11453.rst 5e-6 400 11453 0 MATRES abc 0 1 1 1 0 -10 11
# python main_pair.py 0511pm-lr5e-6-b20-gpu9942-loss0-dataMATRES-accum1-marker@**@-pair1-acr0-tmarker1-td1-dpn1-mask0.rst 5e-6 400 11453 0 MATRES abc 0 1 1 1 0 -10 11

if params['transformers_model'][-5:] == "large":
    params['emb_size'] = 1024
elif params['transformers_model'][-4:] == "base":
    params['emb_size'] = 768
else:
    print("emb_size is neither 1024 nor 768? ...")
    
set_seed(params['seed'])
rst_file_name = params['rst_file_name']

In [9]:
print("Processing " + params['dataset'] + " dataset...")
t0 = time.time()
if params['dataset'] == "IC":
    dir_name = "./IC/IC_Processed/"
    #max_sent_len = 193
elif params['dataset'] == "HiEve":
    dir_name = "./hievents_v2/processed/"
    #max_sent_len = 155
elif params['dataset'] == "MATRES":
    dir_name = ""
else:
    print("Not supporting this dataset yet!")
    
tokenizer = AutoTokenizer.from_pretrained(params['transformers_model'])   
if acronym:
    special_tokens_dict = {'additional_special_tokens': 
                           [' [futuperfsimp]',' [futucont]',' [futuperfcont]',' [futusimp]', ' [pastcont]', ' [pastperfcont]', ' [pastperfsimp]', ' [pastsimp]', ' [prescont]', ' [presperfcont]', ' [presperfsimp]', ' [pressimp]', ' [futuperfsimppass]',' [futucontpass]',' [futuperfcontpass]',' [futusimppass]', ' [pastcontpass]', ' [pastperfcontpass]', ' [pastperfsimppass]', ' [pastsimppass]', ' [prescontpass]', ' [presperfcontpass]', ' [presperfsimppass]', ' [pressimppass]', ' [none]'
                           ]}
    spec_toke_list = []
    for t in special_tokens_dict['additional_special_tokens']:
        spec_toke_list.append(" [/" + t[2:])
    special_tokens_dict['additional_special_tokens'] += spec_toke_list
    num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
    model = AutoModel.from_pretrained(params['transformers_model'])
    model.resize_token_embeddings(len(tokenizer))
else:
    model = AutoModel.from_pretrained(params['transformers_model'])
params['model'] = model
debug = params['debug']
if debug:
    params['epochs'] = 1

Processing MATRES dataset...


Some weights of the model checkpoint at google/bigbird-roberta-large were not used when initializing BigBirdModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BigBirdModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [52]:
t_marker

1

In [60]:
doc_id = -1
features_train = []
features_valid = []
features_test = []
t0 = time.time()
relation_stats = {0: 0, 1: 0, 2: 0, 3: 0}
t_marker = params['t_marker']
# 2: will [futusimp] begin [/futusimp]
# 1: will @ * Future Simple * begin @ 

max_len = 0
sent_num = 0
pair_num = 0
test_labels = []
context_len = {}
timeline_input = []
for fname in tqdm.tqdm(eiid_pair_to_label.keys()):
    file_name = fname + ".tml"
    if file_name in onlyfiles_TB:
        dir_name = mypath_TB
    elif file_name in onlyfiles_AQ:
        dir_name = mypath_AQ
    elif file_name in onlyfiles_PL:
        dir_name = mypath_PL
    else:
        continue

    my_dict = tml_reader(dir_name, file_name, tokenizer) 
    
    for (eiid1, eiid2) in eiid_pair_to_label[fname].keys():
        pair_num += 1
        event_pos = []
        event_pos_end = []
        relations = []
        TokenIDs = [65]
        if eiid1 == 0:
            print('error', fname, eiid1)
            continue
        x = my_dict["eiid_dict"][eiid1]["eID"] # eID
        y = my_dict["eiid_dict"][eiid2]["eID"]
        x_sent_id = my_dict["event_dict"][x]["sent_id"]
        y_sent_id = my_dict["event_dict"][y]["sent_id"]
        reverse = False
        if x_sent_id > y_sent_id:
            reverse = True
            x = my_dict["eiid_dict"][eiid2]["eID"]
            y = my_dict["eiid_dict"][eiid1]["eID"]
            x_sent_id = my_dict["event_dict"][x]["sent_id"]
            y_sent_id = my_dict["event_dict"][y]["sent_id"]
        elif x_sent_id == y_sent_id:
            x_position = my_dict["event_dict"][x]["_subword_id"]
            y_position = my_dict["event_dict"][y]["_subword_id"]
            if x_position > y_position:
                reverse = True
                x = my_dict["eiid_dict"][eiid2]["eID"]
                y = my_dict["eiid_dict"][eiid1]["eID"]
        x_sent = my_dict["sentences"][x_sent_id]["_subword_to_ID"]
        y_sent = my_dict["sentences"][y_sent_id]["_subword_to_ID"]
        # This guarantees that trigger x is always before trigger y in narrative order

        context_start_sent_id = max(x_sent_id-1, 0)
        context_end_sent_id = min(y_sent_id+2, len(my_dict["sentences"]))
        c_len = context_end_sent_id - context_start_sent_id
        if c_len in context_len.keys():
            context_len[c_len] += 1
        else:
            context_len[c_len] = 1
        sent_num += c_len
        
        if params['td'] == 1:
            x_sent, offset_x, new_start_x, new_end_x = add_tense_info(x_sent, my_dict["event_dict"][x]['tense'], my_dict['event_dict'][x]['_subword_id'], my_dict["event_dict"][x]['mention'], 2589, 1736)
        else:
            x_sent, offset_x, new_start_x, new_end_x = x_sent, 0, my_dict['event_dict'][x]['_subword_id'], my_dict['event_dict'][x]['_subword_id'] + len(tokenizer.encode(my_dict["event_dict"][x]['mention'])) - 2
            
        if x_sent_id != y_sent_id:
            if params['td'] == 1:
                y_sent, offset_y, new_start_y, new_end_y = add_tense_info(y_sent, my_dict["event_dict"][y]['tense'], my_dict['event_dict'][y]['_subword_id'], my_dict["event_dict"][y]['mention'], 1404, 5400)
            else:
                y_sent, offset_y, new_start_y, new_end_y = y_sent, 0, my_dict['event_dict'][y]['_subword_id'], my_dict['event_dict'][y]['_subword_id'] + len(tokenizer.encode(my_dict["event_dict"][y]['mention'])) - 2
            for sid in range(context_start_sent_id, context_end_sent_id):
                if sid == x_sent_id:
                    event_pos.append(new_start_x + len(TokenIDs) - 1)
                    event_pos_end.append(new_end_x + len(TokenIDs) - 1)
                    TokenIDs += x_sent[1:]
                elif sid == y_sent_id:
                    event_pos.append(new_start_y + len(TokenIDs) - 1)
                    event_pos_end.append(new_end_y + len(TokenIDs) - 1)
                    TokenIDs += y_sent[1:]
                else:
                    TokenIDs += my_dict["sentences"][sid]["_subword_to_ID"][1:]
        else:
            if params['td'] == 1:
                y_sent, offset_y, new_start_y, new_end_y = add_tense_info(x_sent, my_dict["event_dict"][y]['tense'], my_dict['event_dict'][y]['_subword_id'] + offset_x, my_dict["event_dict"][y]['mention'], 1404, 5400)
            else:
                y_sent, offset_y, new_start_y, new_end_y = y_sent, 0, my_dict['event_dict'][y]['_subword_id'], my_dict['event_dict'][y]['_subword_id'] + len(tokenizer.encode(my_dict["event_dict"][y]['mention'])) - 2
            for sid in range(context_start_sent_id, context_end_sent_id):
                if sid == y_sent_id:
                    event_pos.append(new_start_x + len(TokenIDs) - 1)
                    event_pos_end.append(new_end_x + len(TokenIDs) - 1)
                    event_pos.append(new_start_y + len(TokenIDs) - 1)
                    event_pos_end.append(new_end_y + len(TokenIDs) - 1)
                    TokenIDs += y_sent[1:]
                else:
                    TokenIDs += my_dict["sentences"][sid]["_subword_to_ID"][1:]
                    
        if reverse:
            event_pos = reverse_num(event_pos)
            event_pos_end = reverse_num(event_pos_end)
            
        xy = eiid_pair_to_label[fname][(eiid1, eiid2)]
        
        relations.append(xy)
        relation_stats[xy] += 1
        if len(TokenIDs) > max_len:
            max_len = len(TokenIDs)
        
        if debug or pair_num < 5:
            print("first event of the pair:", tokenizer.decode(TokenIDs[event_pos[0]:event_pos_end[0]]))
            print("second event of the pair:", tokenizer.decode(TokenIDs[event_pos[1]:event_pos_end[1]]))
            print("TokenIDs:", tokenizer.decode(TokenIDs))
        
        if params['nd']:
            syn_0 = replace_with_syn(tokenizer.decode(TokenIDs[event_pos[0]:event_pos_end[0]]))
            syn_1 = replace_with_syn(tokenizer.decode(TokenIDs[event_pos[1]:event_pos_end[1]]))
            if len(syn_0) > 0:
                TokenIDs = TokenIDs[0:event_pos[0]] + tokenizer.encode(syn_0[0])[1:-1] + TokenIDs[event_pos_end[0]:]
                prev = event_pos_end[0]
                event_pos_end[0] = event_pos[0] + len(tokenizer.encode(syn_0[0])[1:-1])
                if prev != event_pos_end[0]:
                    offset = event_pos_end[0] - prev
                    event_pos[1] += offset
                    event_pos_end[1] += offset
            if len(syn_1) > 0:
                TokenIDs = TokenIDs[0:event_pos[1]] + tokenizer.encode(syn_1[0])[1:-1] + TokenIDs[event_pos_end[1]:]
                prev = event_pos_end[1]
                event_pos_end[1] = event_pos[1] + len(tokenizer.encode(syn_1[0])[1:-1])
            #assert 1 == 0
        feature = {'input_ids': TokenIDs,
                   'event_pos': event_pos,
                   'event_pos_end': event_pos_end,
                   'event_pair': [[1, 2]],
                   'labels': relations,
                  }
        if file_name in onlyfiles_TB:
            features_train.append(feature)
        elif file_name in onlyfiles_AQ:
            features_valid.append(feature)
        elif file_name in onlyfiles_PL:
            features_test.append(feature)
            test_labels.append(xy)
            timeline_input.append([fname, x, y, xy])
    if debug:
        break
        
elapsed = format_time(time.time() - t0)
print("MATRES Preprocessing took {:}".format(elapsed)) 
print("Temporal Relation Stats:", relation_stats)
print("Total num of pairs:", pair_num)
print("Max length of context:", max_len)
print("Avg num of sentences that context contains:", sent_num/pair_num)
print("Context length stats(unit: sentence): ", context_len)
print("MATRES train valid test pair num:", len(features_train), len(features_valid), len(features_test))
#with open("MATRES_test_timeline_input.json", 'w') as f:
#    json.dump(timeline_input, f)
#    assert 0 == 1
    
#output_file = open('test_labels.txt', 'w')
#for label in test_labels:
#    output_file.write(str(label) + '\n')
#output_file.close()
#if debug:
#    assert 0 == 1

  1%|█▏                                                                                                                                                                      | 2/274 [00:00<00:51,  5.28it/s]

first event of the pair: predicted
second event of the pair: tried
TokenIDs: [CLS] For his part, Fidel Castro is the ultimate political survivor.[SEP] People have @ * Present Perfect Simple * predicted @ his demise so many times, and the US has # ~ Present Perfect Simple ~ tried # to hasten it on several occasions.[SEP] Time and again, he endures.[SEP]
first event of the pair: predicted
second event of the pair: hasten
TokenIDs: [CLS] For his part, Fidel Castro is the ultimate political survivor.[SEP] People have @ * Present Perfect Simple * predicted @ his demise so many times, and the US has tried to # ~ None ~ hasten # it on several occasions.[SEP] Time and again, he endures.[SEP]
first event of the pair: tried
second event of the pair: hasten
TokenIDs: [CLS] For his part, Fidel Castro is the ultimate political survivor.[SEP] People have predicted his demise so many times, and the US has @ * Present Perfect Simple * tried @ to # ~ None ~ hasten # it on several occasions.[SEP] Time a

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 274/274 [01:01<00:00,  4.49it/s]

MATRES Preprocessing took 0:01:01
Temporal Relation Stats: {0: 6874, 1: 4570, 2: 470, 3: 1644}
Total num of pairs: 13558
Max length of context: 245
Avg num of sentences that context contains: 3.562546098244579
Context length stats(unit: sentence):  {3: 4911, 4: 7981, 2: 554, 5: 106, 1: 6}
MATRES train valid test pair num: 6336 6404 818


In [59]:
params['nd'] = False

In [ ]:
1

# unit test text processing

In [26]:
(dir_name, file_name, tokenizer)

('./MATRES_old/te3-platinum/',
 'CNN_20130321_821.tml',
 BigBirdTokenizerFast(name_or_path='google/bigbird-roberta-large', vocab_size=50358, model_max_length=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=True)}))

In [27]:
my_dict = {}
my_dict["event_dict"] = {}
my_dict["eiid_dict"] = {}
my_dict["doc_id"] = file_name.replace(".tml", "") 
# e.g., file_name = "ABC19980108.1830.0711.tml"
# dir_name = '/shared/why16gzl/logic_driven/EMNLP-2020/MATRES/TBAQ-cleaned/TimeBank/'
tree = ET.parse(dir_name + file_name)
root = tree.getroot()
MY_STRING = str(ET.tostring(root))
# ================================================
# Load the lines involving event information first
# ================================================
event_id_why = 0
for makeinstance in root.findall('MAKEINSTANCE'):
    instance_str = str(ET.tostring(makeinstance)).split(" ")
    try:
        assert instance_str[3].split("=")[0] == "eventID"
        assert instance_str[2].split("=")[0] == "eiid"
        eiid = int(instance_str[2].split("=")[1].replace("\"", "")[2:])
        eID = instance_str[3].split("=")[1].replace("\"", "")
    except:
        for i in instance_str:
            if i.split("=")[0] == "eventID":
                eID = i.split("=")[1].replace("\"", "")
            if i.split("=")[0] == "eiid":
                eiid = int(i.split("=")[1].replace("\"", "")[2:])
    # Not all document in the dataset contributes relation pairs in MATRES
    # Not all events in a document constitute relation pairs in MATRES

    if my_dict["doc_id"] in eiid_to_event_trigger.keys():
        if eiid in eiid_to_event_trigger[my_dict["doc_id"]].keys():
            event_id_why += 1
            my_dict["event_dict"][eID] = {"eiid": eiid, "mention": eiid_to_event_trigger[my_dict["doc_id"]][eiid], "event_id_why": event_id_why}
            my_dict["eiid_dict"][eiid] = {"eID": eID}

# ==================================
#              Load Text
# ==================================
start = MY_STRING.find("<TEXT>") + 6
end = MY_STRING.find("</TEXT>")
MY_TEXT = MY_STRING[start:end]
while MY_TEXT[0] == " ":
    MY_TEXT = MY_TEXT[1:]
MY_TEXT = MY_TEXT.replace("\\n", " ")
MY_TEXT = MY_TEXT.replace("\\'", "\'")
MY_TEXT = MY_TEXT.replace("  ", " ")
MY_TEXT = MY_TEXT.replace(" ...", "...")


In [28]:
# ========================================================
#    Load position of events, in the meantime replacing 
#    "<EVENT eid="e1" class="OCCURRENCE">turning</EVENT>"
#    with "turning"
# ========================================================
while MY_TEXT.find("<") != -1:
    start = MY_TEXT.find("<")
    end = MY_TEXT.find(">")
    if MY_TEXT[start + 1] == "E":
        event_description = MY_TEXT[start:end].split(" ")
        print(event_description)
        eID = (event_description[1].split("="))[1].replace("\"", "")
        MY_TEXT = MY_TEXT[:start] + MY_TEXT[(end + 1):]
        if eID in my_dict["event_dict"].keys():
            my_dict["event_dict"][eID]["start_char"] = start # loading position of events
    else:
        MY_TEXT = MY_TEXT[:start] + MY_TEXT[(end + 1):]


['<EVENT', 'class="OCCURRENCE"', 'eid="e1"']
['<EVENT', 'class="STATE"', 'eid="e4"']
['<EVENT', 'class="OCCURRENCE"', 'eid="e5"']
['<EVENT', 'class="OCCURRENCE"', 'eid="e8"']
['<EVENT', 'eid="e1000"', 'class="I_ACTION"']
['<EVENT', 'class="I_ACTION"', 'eid="e1002"']
['<EVENT', 'class="OCCURRENCE"', 'eid="e1003"']
['<EVENT', 'class="I_ACTION"', 'eid="e20"']


In [20]:
event_description

['<EVENT', 'class="REPORTING"', 'eid="e55"']

In [ ]:
# =====================================
# Enter the routine for text processing
# =====================================

my_dict["doc_content"] = MY_TEXT
my_dict["sentences"] = []
my_dict["relation_dict"] = {}
sent_tokenized_text = sent_tokenize(my_dict["doc_content"])
#sent_tokenized_text = []
#tense_res = tense_getter(MY_TEXT)
#for sentence in tense_res['sentences']:
#    sent_tokenized_text.append(sentence[0])

sent_span = tokenized_to_origin_span(my_dict["doc_content"], sent_tokenized_text)
end_pos = [1]
for count_sent, sent in enumerate(sent_tokenized_text):
    sent_dict = {}
    sent_dict["sent_id"] = count_sent
    sent_dict["content"] = sent
    sent_dict["sent_start_char"] = sent_span[count_sent][0]
    sent_dict["sent_end_char"] = sent_span[count_sent][1]
    sent_dict["tense_list"] = tense_getter(sent)

    spacy_token = nlp(sent_dict["content"])
    sent_dict["tokens"] = []
    sent_dict["pos"] = []
    # spaCy-tokenized tokens & Part-Of-Speech Tagging
    for token in spacy_token:
        sent_dict["tokens"].append(token.text)
        sent_dict["pos"].append(token.pos_)
    sent_dict["token_span_SENT"] = tokenized_to_origin_span(sent, sent_dict["tokens"])
    sent_dict["token_span_DOC"] = span_SENT_to_DOC(sent_dict["token_span_SENT"], sent_dict["sent_start_char"])

    # huggingface tokenizer
    sent_dict["_subword_to_ID"], sent_dict["_subwords"], \
    sent_dict["_subword_span_SENT"], sent_dict["_subword_map"] = \
    transformers_list(sent_dict["content"], tokenizer, sent_dict["tokens"], sent_dict["token_span_SENT"])

    if count_sent == 0:
        end_pos.append(len(sent_dict["_subword_to_ID"]))
    else:
        end_pos.append(end_pos[-1] + len(sent_dict["_subword_to_ID"]) - 1)

    sent_dict["_subword_span_DOC"] = \
    span_SENT_to_DOC(sent_dict["_subword_span_SENT"], sent_dict["sent_start_char"])

    sent_dict["_subword_pos"] = []
    for token_id in sent_dict["_subword_map"]:
        if token_id == -1 or token_id is None:
            sent_dict["_subword_pos"].append("None")
        else:
            sent_dict["_subword_pos"].append(sent_dict["pos"][token_id])

    my_dict["sentences"].append(sent_dict)
    count_sent += 1

my_dict['end_pos'] = end_pos
# Add sent_id as an attribute of event
for event_id, event_dict in my_dict["event_dict"].items():
    my_dict["event_dict"][event_id]["sent_id"] = sent_id = \
    sent_id_lookup(my_dict, event_dict["start_char"])
    my_dict["event_dict"][event_id]["token_id"] = \
    id_lookup(my_dict["sentences"][sent_id]["token_span_DOC"], event_dict["start_char"])
    my_dict["event_dict"][event_id]["_subword_id"] = \
    id_lookup(my_dict["sentences"][sent_id]["_subword_span_DOC"], event_dict["start_char"]) + 1 
    # updated on Mar 20, 2021
    #my_dict["event_dict"][event_id]["tense"] = tense_finder(tense_res['sentences'][sent_id][1], event_dict["start_char"] - my_dict['sentences'][sent_id]["sent_start_char"])
    # updated on Feb 21, 2021
    my_dict["event_dict"][event_id]["tense"] = tense_finder(my_dict['sentences'][sent_id]["tense_list"], event_dict["start_char"] - my_dict['sentences'][sent_id]["sent_start_char"])
    # updated on Oct 24, 2022, because of change of tense identification service


In [24]:
dir_name

'./MATRES_old/te3-platinum/'

In [32]:
features_train[0]

{'input_ids': [65,
  1215,
  566,
  737,
  112,
  42421,
  21294,
  419,
  363,
  8814,
  2065,
  23547,
  114,
  66,
  4481,
  524,
  2589,
  1736,
  21763,
  16475,
  17528,
  1736,
  11102,
  2589,
  566,
  25504,
  624,
  968,
  1762,
  112,
  391,
  363,
  1395,
  569,
  1404,
  5400,
  21763,
  16475,
  17528,
  5400,
  3189,
  1404,
  385,
  488,
  26501,
  441,
  420,
  1912,
  12533,
  114,
  66,
  3963,
  391,
  858,
  112,
  440,
  987,
  1043,
  114,
  66],
 'event_pos': [22, 40],
 'event_pos_end': [23, 41],
 'event_pair': [[1, 2]],
 'labels': [0]}

# model

In [33]:
params = {'transformers_model': 'google/bigbird-roberta-large',
          'dataset': 'MATRES',   # 'HiEve', 'IC', 'MATRES' 
          'testdata': 'PRED', # MATRES / MATRES_nd / TDD / PRED / None; None means training mode
          'block_size': 64,
          'add_loss': 0, 
          'batch_size': 1,    # 6 works on 48G gpu
          'epochs': 40,
          'learning_rate': 5e-6,    # subject to change
          'seed': 0,
          'gpu_id': '11453',    # subject to change
          'debug': 0,
          'rst_file_name': '0511pm-lr5e-6-b20-gpu9942-loss0-dataMATRES-accum1-marker@**@-pair1-acr0-tmarker1-td1-dpn1-mask0.rst',    # subject to change
          'mask_in_input_ids': mask_in_input_ids,
          'mask_in_input_mask': mask_in_input_mask,
          'marker': 'abc', 
          'tense_acron': 0, # 1 (acronym of tense) or 0 (original tense)
          't_marker': 1, # 2 (trigger enclosed by special tokens) or 1 (tense enclosed by **)
          'td': 1, # 0 (no tense detection) or 1 (tense detection, add tense info)
          'dpn': 0, # 1 if use DPN; else 0
          'lambda_1': -10, # lower bound * 10
          'lambda_2': 11, # upper bound * 10
          'f1_metric': f1_metric, 
         }

In [36]:
if acronym:
    special_tokens_dict = {'additional_special_tokens': 
                           [' [futuperfsimp]',' [futucont]',' [futuperfcont]',' [futusimp]', ' [pastcont]', ' [pastperfcont]', ' [pastperfsimp]', ' [pastsimp]', ' [prescont]', ' [presperfcont]', ' [presperfsimp]', ' [pressimp]', ' [futuperfsimppass]',' [futucontpass]',' [futuperfcontpass]',' [futusimppass]', ' [pastcontpass]', ' [pastperfcontpass]', ' [pastperfsimppass]', ' [pastsimppass]', ' [prescontpass]', ' [presperfcontpass]', ' [presperfsimppass]', ' [pressimppass]', ' [none]'
                           ]}
    spec_toke_list = []
    for t in special_tokens_dict['additional_special_tokens']:
        spec_toke_list.append(" [/" + t[2:])
    special_tokens_dict['additional_special_tokens'] += spec_toke_list
    num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
    model = AutoModel.from_pretrained(params['transformers_model'])
    model.resize_token_embeddings(len(tokenizer))
else:
    model = AutoModel.from_pretrained(params['transformers_model'])
params['model'] = model

Some weights of the model checkpoint at google/bigbird-roberta-large were not used when initializing BigBirdModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BigBirdModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [44]:
train_dataloader = DataLoader(features_train, batch_size=params['batch_size'], shuffle=True, collate_fn=collate_fn, drop_last=True)
valid_dataloader = DataLoader(features_valid, batch_size=params['batch_size'], shuffle=False, collate_fn=collate_fn, drop_last=False)
test_dataloader = DataLoader(features_test, batch_size=params['batch_size'], shuffle=False, collate_fn=collate_fn, drop_last=False)

In [43]:
OnePassModel = transformers_mlp_cons(params)
OnePassModel.to(cuda)
OnePassModel.zero_grad()
print("# of parameters:", count_parameters(OnePassModel))

# of parameters: 364635144


In [39]:
params['cuda'] = cuda # not included in config file

In [47]:
best_PATH = "./"
best_1 = -0.4
best_2 = 0.6

In [42]:
params['emb_size'] = 1024

In [50]:
mem_exp_test = exp(cuda, OnePassModel, params['epochs'], params['learning_rate'], 
                   train_dataloader, valid_dataloader, test_dataloader, 
                   params['dataset'], best_PATH, None, params['dpn'], model_name, None, [best_1, best_2])


In [49]:
model_name = rst_file_name.replace(".rst", "")
model_name

'0511pm-lr5e-6-b20-gpu9942-loss0-dataMATRES-accum1-marker@**@-pair1-acr0-tmarker1-td1-dpn1-mask0'

In [51]:
mem_exp_test.train()

Attention type 'block_sparse' is not possible if sequence_length: 120 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...



======== Epoch 1 / 40 ========
Training...
  Batch    40  of  6,332.    Elapsed: 0:00:12.
  Batch    80  of  6,332.    Elapsed: 0:00:23.
  Batch   120  of  6,332.    Elapsed: 0:00:35.


KeyboardInterrupt: 